### Import libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import numpy as np
import cv2
import os

### Preprocess images

#### Convert images into grayscale, invert colors and dilate them

In [4]:
dataDir = "../data_copy_copy/"

dirList = os.listdir(dataDir)

for directory in dirList:
    path = os.path.join(dataDir, directory)

    for fileName in os.listdir(path):
        imagePath = os.path.join(path, fileName)

        image = cv2.imread(imagePath)

        grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        kernel = np.ones((3, 3), np.uint8)

        dilatedImage = cv2.dilate(grayscale, kernel, iterations=1)

        folder = os.path.join(dataDir, directory)

        imagePath = folder + "/" + fileName + "again.jpg"

        cv2.imwrite(imagePath, dilatedImage)


### Load and split data

In [6]:
ds_train = keras.utils.image_dataset_from_directory(
    directory = "../data_copy_copy/",
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=64,
    image_size=(45, 45),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="training"
)

ds_val = keras.utils.image_dataset_from_directory(
    directory = "../data_copy_copy/",
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=64,
    image_size=(45, 45),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="validation"
)

ds_test = ds_val.take(290)
ds_val = ds_val.skip(290)

Found 370930 files belonging to 16 classes.
Using 296744 files for training.
Found 370930 files belonging to 16 classes.
Using 74186 files for validation.


In [7]:
print('Batches for testing -->', ds_test.cardinality())
print('Batches for validating -->', ds_val.cardinality())

Batches for testing --> tf.Tensor(290, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(870, shape=(), dtype=int64)


### Create model

In [8]:

img_height = 45
img_width = 45
num_classes = 16
img_channels = 1

model = keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width, img_channels)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

### Compile model

In [9]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Train model

In [10]:
model.fit(ds_train, epochs=15, validation_data=ds_val)

Epoch 1/15


2022-04-27 18:13:38.886167: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8303


4637/4637 [==============================] - 36s 7ms/step - loss: 0.1528 - accuracy: 0.9657 - val_loss: 0.0531 - val_accuracy: 0.9832
Epoch 2/15
4637/4637 [==============================] - 22s 5ms/step - loss: 0.0515 - accuracy: 0.9834 - val_loss: 0.0358 - val_accuracy: 0.9891
Epoch 3/15
4637/4637 [==============================] - 23s 5ms/step - loss: 0.0407 - accuracy: 0.9867 - val_loss: 0.0366 - val_accuracy: 0.9888
Epoch 4/15
4637/4637 [==============================] - 21s 5ms/step - loss: 0.0348 - accuracy: 0.9885 - val_loss: 0.0287 - val_accuracy: 0.9904
Epoch 5/15
4637/4637 [==============================] - 20s 4ms/step - loss: 0.0317 - accuracy: 0.9897 - val_loss: 0.0254 - val_accuracy: 0.9924
Epoch 6/15
4637/4637 [==============================] - 20s 4ms/step - loss: 0.0285 - accuracy: 0.9906 - val_loss: 0.0235 - val_accuracy: 0.9922
Epoch 7/15
4637/4637 [==============================] - 20s 4ms/step - loss: 0.0265 - accuracy: 0.9914 - val_loss: 0.0210 - val_accuracy: 0.9

### Evaluate model on test set

In [11]:
model.evaluate(ds_test)

290/290 [==============================] - 1s 3ms/step - loss: 0.0152 - accuracy: 0.9956


[0.015204863622784615, 0.9955819249153137]

### Save model

In [12]:
model.save("../models/model2")

2022-04-27 18:19:16.125794: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/model2/assets
